In [50]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        
import math

In [51]:
data_ref = pd.read_excel("Rw_ref_curves.xlsx", sheet_name="Ref")
PW_parametric = pd.read_csv("PW_parstud_combined_4phyt.csv")
PW_parametric.head()

,No,l,h,g,Abs_mat,w,ol,Measured_Rw,DoubleGlassRw,CompositeClosedRw,...,abs_500,abs_630,abs_800,abs_1000,abs_1250,abs_1600,abs_2000,abs_2500,abs_3150,Unnamed: 42
0,1,2400,1300,200.0,No,200.0,2000.0,17,35,NaN,...,0.64,0.78,0.78,0.82,0.81,0.74,0.7,0.65,0.64,NaN
1,2,2400,1300,200.0,Yes,25.0,2350.0,31,35,NaN,...,0.64,0.78,0.78,0.82,0.81,0.74,0.7,0.65,0.64,NaN
2,3,2400,1300,200.0,Yes,50.0,2300.0,28,35,NaN,...,0.64,0.78,0.78,0.82,0.81,0.74,0.7,0.65,0.64,NaN
3,4,2400,1300,200.0,Yes,100.0,2200.0,26,35,NaN,...,0.64,0.78,0.78,0.82,0.81,0.74,0.7,0.65,0.64,NaN
4,5,2400,1300,200.0,Yes,200.0,2000.0,23,35,NaN,...,0.64,0.78,0.78,0.82,0.81,0.74,0.7,0.65,0.64,NaN


In [52]:
l = PW_parametric['l']
h = PW_parametric['h']
w = PW_parametric['w']
g = PW_parametric['g']
OL = PW_parametric['ol']
d = OL + w
PW_parametric['d'] = d
Abs_mat = PW_parametric['Abs_mat']
Measured_Rw = PW_parametric['Measured_Rw']
PW_parametric['SingleGlassRw'] = 25
SingleGlassRw = PW_parametric['SingleGlassRw']
DoubleGlassRw = PW_parametric['DoubleGlassRw']

PW_parametric.head()

,No,l,h,g,Abs_mat,w,ol,Measured_Rw,DoubleGlassRw,CompositeClosedRw,...,abs_800,abs_1000,abs_1250,abs_1600,abs_2000,abs_2500,abs_3150,Unnamed: 42,d,SingleGlassRw
0,1,2400,1300,200.0,No,200.0,2000.0,17,35,NaN,...,0.78,0.82,0.81,0.74,0.7,0.65,0.64,NaN,2200.0,25
1,2,2400,1300,200.0,Yes,25.0,2350.0,31,35,NaN,...,0.78,0.82,0.81,0.74,0.7,0.65,0.64,NaN,2375.0,25
2,3,2400,1300,200.0,Yes,50.0,2300.0,28,35,NaN,...,0.78,0.82,0.81,0.74,0.7,0.65,0.64,NaN,2350.0,25
3,4,2400,1300,200.0,Yes,100.0,2200.0,26,35,NaN,...,0.78,0.82,0.81,0.74,0.7,0.65,0.64,NaN,2300.0,25
4,5,2400,1300,200.0,Yes,200.0,2000.0,23,35,NaN,...,0.78,0.82,0.81,0.74,0.7,0.65,0.64,NaN,2200.0,25


In [53]:
print(l.describe())
print(h.describe())
print(g.describe())
print(w.describe())
print(OL.describe())
print(Abs_mat.value_counts())


count      47.000000
mean     2260.851064
std      1396.982509
min       740.000000
25%      1320.000000
50%      1900.000000
75%      2420.000000
max      5600.000000
Name: l, dtype: float64
count      47.000000
mean     1355.319149
std       377.525416
min       550.000000
25%      1300.000000
50%      1350.000000
75%      1350.000000
max      2400.000000
Name: h, dtype: float64
count     47.000000
mean     207.485106
std       90.330596
min       85.000000
25%      145.000000
50%      200.000000
75%      205.000000
max      560.000000
Name: g, dtype: float64
count      47.000000
mean      520.463830
std       348.308129
min        25.000000
25%       213.650000
50%       560.000000
75%       787.500000
max      1250.000000
Name: w, dtype: float64
count      47.000000
mean     1126.421277
std      1260.205088
min       100.000000
25%       220.000000
50%       700.000000
75%      1290.900000
max      5365.000000
Name: ol, dtype: float64
Yes    32
No     15
Name: Abs_mat, dtype: int64

In [54]:
Q0 = 2
K0 = 1
Q1 = 1.2088
K1 = 0.2288

In [55]:
PW_parametric['sigSA_Yabs_100'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_100']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_125'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_125']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_160'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_160']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_200'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_200']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_250'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_250']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_315'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_315']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_400'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_400']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_500'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_500']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_630'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_630']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_800'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_800']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_1000'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_1000']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_1250'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_1250']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_1600'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_1600']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_2000'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_2000']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_2500'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_2500']) + (2*w*h/1000000*0.9)
PW_parametric['sigSA_Yabs_3150'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*PW_parametric['abs_3150']) + (2*w*h/1000000*0.9)

PW_parametric['sigSA_Nabs'] = ((2*l/1000*h/1000 - 2*w*h/1000000)*0.03) + ((2*g/1000*l/1000+2*g/1000*h/1000)*0.05) + (2*w*h/1000000*0.9)

PW_parametric['sigSA_per_sigS_Yabs_100'] = PW_parametric['sigSA_Yabs_100']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_125'] = PW_parametric['sigSA_Yabs_125']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_160'] = PW_parametric['sigSA_Yabs_160']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_200'] = PW_parametric['sigSA_Yabs_200']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_250'] = PW_parametric['sigSA_Yabs_250']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_315'] = PW_parametric['sigSA_Yabs_315']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_400'] = PW_parametric['sigSA_Yabs_400']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_500'] = PW_parametric['sigSA_Yabs_500']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_630'] = PW_parametric['sigSA_Yabs_630']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_800'] = PW_parametric['sigSA_Yabs_800']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_1000'] = PW_parametric['sigSA_Yabs_1000']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_1250'] = PW_parametric['sigSA_Yabs_1250']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_1600'] = PW_parametric['sigSA_Yabs_1600']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_2000'] = PW_parametric['sigSA_Yabs_2000']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_2500'] = PW_parametric['sigSA_Yabs_2500']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))
PW_parametric['sigSA_per_sigS_Yabs_3150'] = PW_parametric['sigSA_Yabs_3150']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))

PW_parametric['sigSA_per_sigS_Nabs'] = PW_parametric['sigSA_Nabs']/((2*l/1000*h/1000 - 2*w*h/1000000)+(2*g/1000*l/1000+2*g/1000*h/1000)+(2*w*h/1000000))

PW_parametric.head()

,No,l,h,g,Abs_mat,w,ol,Measured_Rw,DoubleGlassRw,CompositeClosedRw,...,sigSA_per_sigS_Yabs_500,sigSA_per_sigS_Yabs_630,sigSA_per_sigS_Yabs_800,sigSA_per_sigS_Yabs_1000,sigSA_per_sigS_Yabs_1250,sigSA_per_sigS_Yabs_1600,sigSA_per_sigS_Yabs_2000,sigSA_per_sigS_Yabs_2500,sigSA_per_sigS_Yabs_3150,sigSA_per_sigS_Nabs
0,1,2400,1300,200.0,No,200.0,2000.0,17,35,NaN,...,0.205544,0.232383,0.232383,0.240052,0.238135,0.224715,0.217047,0.207461,0.205544,0.092435
1,2,2400,1300,200.0,Yes,25.0,2350.0,31,35,NaN,...,0.154268,0.181108,0.181108,0.188776,0.186859,0.173439,0.165771,0.156185,0.154268,0.041159
2,3,2400,1300,200.0,Yes,50.0,2300.0,28,35,NaN,...,0.161593,0.188433,0.188433,0.196101,0.194184,0.180764,0.173096,0.163510,0.161593,0.048484
3,4,2400,1300,200.0,Yes,100.0,2200.0,26,35,NaN,...,0.176244,0.203083,0.203083,0.210751,0.208834,0.195415,0.187746,0.178161,0.176244,0.063135
4,5,2400,1300,200.0,Yes,200.0,2000.0,23,35,NaN,...,0.205544,0.232383,0.232383,0.240052,0.238135,0.224715,0.217047,0.207461,0.205544,0.092435


In [56]:
PW_parametric['Diffration_Loss'] = -10*np.log10(w*h/1000000*((Q1*g/1000/(d/1000)/(4*math.pi*(d/1000)**2))))

PW_parametric['Reverberation_Loss_Yabs_100'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_100'])/PW_parametric['sigSA_Yabs_100'])
PW_parametric['Reverberation_Loss_Yabs_125'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_125'])/PW_parametric['sigSA_Yabs_125'])
PW_parametric['Reverberation_Loss_Yabs_160'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_160'])/PW_parametric['sigSA_Yabs_160'])
PW_parametric['Reverberation_Loss_Yabs_200'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_200'])/PW_parametric['sigSA_Yabs_200'])
PW_parametric['Reverberation_Loss_Yabs_250'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_250'])/PW_parametric['sigSA_Yabs_250'])
PW_parametric['Reverberation_Loss_Yabs_315'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_315'])/PW_parametric['sigSA_Yabs_315'])
PW_parametric['Reverberation_Loss_Yabs_400'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_400'])/PW_parametric['sigSA_Yabs_400'])
PW_parametric['Reverberation_Loss_Yabs_500'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_500'])/PW_parametric['sigSA_Yabs_500'])
PW_parametric['Reverberation_Loss_Yabs_630'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_630'])/PW_parametric['sigSA_Yabs_630'])
PW_parametric['Reverberation_Loss_Yabs_800'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_800'])/PW_parametric['sigSA_Yabs_800'])
PW_parametric['Reverberation_Loss_Yabs_1000'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_1000'])/PW_parametric['sigSA_Yabs_1000'])
PW_parametric['Reverberation_Loss_Yabs_1250'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_1250'])/PW_parametric['sigSA_Yabs_1250'])
PW_parametric['Reverberation_Loss_Yabs_1600'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_1600'])/PW_parametric['sigSA_Yabs_1600'])
PW_parametric['Reverberation_Loss_Yabs_2000'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_2000'])/PW_parametric['sigSA_Yabs_2000'])
PW_parametric['Reverberation_Loss_Yabs_2500'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_2500'])/PW_parametric['sigSA_Yabs_2500'])
PW_parametric['Reverberation_Loss_Yabs_3150'] = -10*np.log10(w*h/1000000*K1*(1-PW_parametric['sigSA_per_sigS_Yabs_3150'])/PW_parametric['sigSA_Yabs_3150'])

PW_parametric['R_plenum_Yabs_100'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_100']/-10))
PW_parametric['R_plenum_Yabs_125'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_125']/-10))
PW_parametric['R_plenum_Yabs_160'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_160']/-10))
PW_parametric['R_plenum_Yabs_200'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_200']/-10))
PW_parametric['R_plenum_Yabs_250'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_250']/-10))
PW_parametric['R_plenum_Yabs_315'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_315']/-10))
PW_parametric['R_plenum_Yabs_400'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_400']/-10))
PW_parametric['R_plenum_Yabs_500'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_500']/-10))
PW_parametric['R_plenum_Yabs_630'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_630']/-10))
PW_parametric['R_plenum_Yabs_800'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_800']/-10))
PW_parametric['R_plenum_Yabs_1000'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_1000']/-10))
PW_parametric['R_plenum_Yabs_1250'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_1250']/-10))
PW_parametric['R_plenum_Yabs_1600'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_1600']/-10))
PW_parametric['R_plenum_Yabs_2000'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_2000']/-10))
PW_parametric['R_plenum_Yabs_2500'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_2500']/-10))
PW_parametric['R_plenum_Yabs_3150'] = -10*np.log10(10**(PW_parametric['Diffration_Loss']/-10)+10**(PW_parametric['Reverberation_Loss_Yabs_3150']/-10))

PW_parametric['R_plenum_Nabs'] = -10*np.log10(w*h/1000000*((Q1*g/1000/(d/1000)/(4*math.pi*(d/1000)**2)+K1*(1-PW_parametric['sigSA_per_sigS_Nabs'])/PW_parametric['sigSA_Nabs'])))

PW_parametric['R_plenum_Li_1_100'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_100'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_125'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_125'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_160'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_160'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_200'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_200'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_250'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_250'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_315'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_315'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_400'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_400'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_500'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_500'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_630'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_630'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_800'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_800'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_1000'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_1000'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_1250'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_1250'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_1600'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_1600'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_2000'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_2000'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_2500'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_2500'], PW_parametric['R_plenum_Nabs'])
PW_parametric['R_plenum_Li_1_3150'] = np.where(Abs_mat == "Yes", PW_parametric['R_plenum_Yabs_3150'], PW_parametric['R_plenum_Nabs'])

PW_parametric['R_plenum_Li_1_1000']

0     11.184596
1     23.782140
2     20.972594
3     18.346308
4     16.044252
5     21.024242
6     18.331793
7     15.891038
8     13.785285
9     10.556945
10    13.054205
11    10.683007
12    12.610294
13    10.101020
14    12.083627
15    10.479142
16    12.119790
17    10.348159
18    13.154123
19    10.471912
20    12.679786
21     9.053001
22    10.794171
23     9.627798
24    11.190287
25    10.219518
26    13.654808
27    10.309433
28    13.088919
29     7.076597
30     8.367356
31     7.937870
32     9.248018
33     9.436000
34    13.185425
35    10.087812
36    13.030917
37    17.323188
38    19.303987
39    12.908183
40    13.936873
41    13.763361
42    15.116102
43    13.649547
44    15.087201
45    18.719012
46    21.374105
Name: R_plenum_Li_1_1000, dtype: float64

In [57]:
R_geometric_Yu_2 = 20* np.log10(l/w)
PW_parametric['R_geometric_Yu_2'] = R_geometric_Yu_2

PW_parametric.head()

,No,l,h,g,Abs_mat,w,ol,Measured_Rw,DoubleGlassRw,CompositeClosedRw,...,R_plenum_Li_1_500,R_plenum_Li_1_630,R_plenum_Li_1_800,R_plenum_Li_1_1000,R_plenum_Li_1_1250,R_plenum_Li_1_1600,R_plenum_Li_1_2000,R_plenum_Li_1_2500,R_plenum_Li_1_3150,R_geometric_Yu_2
0,1,2400,1300,200.0,No,200.0,2000.0,17,35,NaN,...,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,21.583625
1,2,2400,1300,200.0,Yes,25.0,2350.0,31,35,NaN,...,22.734997,23.563581,23.563581,23.782140,23.728164,23.337567,23.103386,22.797930,22.734997,39.645425
2,3,2400,1300,200.0,Yes,50.0,2300.0,28,35,NaN,...,19.960496,20.760649,20.760649,20.972594,20.920219,20.541847,20.315558,20.021071,19.960496,33.624825
3,4,2400,1300,200.0,Yes,100.0,2200.0,26,35,NaN,...,17.396030,18.146132,18.146132,18.346308,18.296787,17.940111,17.727741,17.452485,17.396030,27.604225
4,5,2400,1300,200.0,Yes,200.0,2000.0,23,35,NaN,...,15.192265,15.863066,15.863066,16.044252,15.999348,15.677517,15.487269,15.242283,15.192265,21.583625


In [58]:
data = PW_parametric[PW_parametric.columns[-17:-1]]
data['Rw'] = 0
col_name = list(data_ref.columns[1:].values)
col_name.append('Rw')
data.columns = col_name
data

C:\Users\Hariz\AppData\Local\Temp\ipykernel_5160\482216958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Rw'] = 0


,100,125,160,200,250,315,400,500,630,800,1000,1250,1600,2000,2500,3150,Rw
0,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,0
1,15.552706,15.802643,17.256062,18.369163,19.393580,20.625248,21.701981,22.734997,23.563581,23.563581,23.782140,23.728164,23.337567,23.103386,22.797930,22.734997,0
2,13.432537,13.640113,14.884276,15.873820,16.807282,17.953014,18.971505,19.960496,20.760649,20.760649,20.972594,20.920219,20.541847,20.315558,20.021071,19.960496,0
3,11.816713,11.972610,12.944005,13.757320,14.552916,15.561439,16.482806,17.396030,18.146132,18.146132,18.346308,18.296787,17.940111,17.727741,17.452485,17.396030,0
4,10.774369,10.879998,11.565100,12.172283,12.793197,13.614666,14.394788,15.192265,15.863066,15.863066,16.044252,15.999348,15.677517,15.487269,15.242283,15.192265,0
5,13.606843,13.817593,15.073085,16.062669,16.989011,18.116117,19.108064,20.061065,20.823653,20.823653,21.024242,20.974731,20.615937,20.400457,20.119074,20.061065,0
6,11.907030,12.066509,13.053278,13.870714,14.662906,15.656504,16.553407,17.431312,18.143297,18.143297,18.331793,18.285223,17.948616,17.747235,17.485196,17.431312,0
7,10.747704,10.855698,11.551155,12.160519,12.777028,13.582372,14.336061,15.094723,15.722986,15.722986,15.891038,15.849455,15.550149,15.372176,15.141890,15.094723,0
8,10.061353,10.127468,10.567603,10.972933,11.400611,11.983944,12.552776,13.145118,13.648764,13.648764,13.785285,13.751438,13.509122,13.366175,13.182539,13.145118,0
9,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,0


In [59]:
def find_rw(data_ref, data_test):
    nama_columns_raw = list(data_ref.columns)
    nama_columns_loop = nama_columns_raw[1:]

    result = []

    for index, row in data_test.iterrows():
        num_row_loop = len(data_ref)
        check_loop = False
        for num_row in range(len(data_ref)):
            num_row_loop -=1
            sum_dev = 0
            if not check_loop:
                for col in nama_columns_loop:
                    if row[col] > data_ref[col][num_row_loop]:
                        if col == 3150:
                            if sum_dev < 32:
                                check_loop = True
                                result.append([num_row_loop,  data_ref['Rw'][num_row_loop]])
                                print("done index: ", index)
                        else:
                            continue
                    else:
                        sum_dev += abs(row[col] - data_ref[col][num_row_loop])
                        if sum_dev > 32:
                            break
                        if col == 3150:
                            if sum_dev < 32:
                                check_loop = True
                                result.append([num_row_loop,  data_ref['Rw'][num_row_loop]])
                                print("done index: ", index)
                        else:
                            continue
            else:
                break
    
    data_test['Rw'] = pd.DataFrame(result)[1]

    return data_test

In [60]:
find_rw(data_ref, data)

done index:  0
done index:  1
done index:  2
done index:  3
done index:  4
done index:  5
done index:  6
done index:  7
done index:  8
done index:  9
done index:  10
done index:  11
done index:  12
done index:  13
done index:  14
done index:  15
done index:  16
done index:  17
done index:  18
done index:  19
done index:  20
done index:  21
done index:  22
done index:  23
done index:  24
done index:  25
done index:  26
done index:  27
done index:  28
done index:  29
done index:  30
done index:  31
done index:  32
done index:  33
done index:  34
done index:  35
done index:  36
done index:  37
done index:  38
done index:  39
done index:  40
done index:  41
done index:  42
done index:  43
done index:  44
done index:  45
done index:  46


C:\Users\Hariz\AppData\Local\Temp\ipykernel_5160\478060343.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['Rw'] = pd.DataFrame(result)[1]


,100,125,160,200,250,315,400,500,630,800,1000,1250,1600,2000,2500,3150,Rw
0,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11.184596,11
1,15.552706,15.802643,17.256062,18.369163,19.393580,20.625248,21.701981,22.734997,23.563581,23.563581,23.782140,23.728164,23.337567,23.103386,22.797930,22.734997,23
2,13.432537,13.640113,14.884276,15.873820,16.807282,17.953014,18.971505,19.960496,20.760649,20.760649,20.972594,20.920219,20.541847,20.315558,20.021071,19.960496,21
3,11.816713,11.972610,12.944005,13.757320,14.552916,15.561439,16.482806,17.396030,18.146132,18.146132,18.346308,18.296787,17.940111,17.727741,17.452485,17.396030,18
4,10.774369,10.879998,11.565100,12.172283,12.793197,13.614666,14.394788,15.192265,15.863066,15.863066,16.044252,15.999348,15.677517,15.487269,15.242283,15.192265,16
5,13.606843,13.817593,15.073085,16.062669,16.989011,18.116117,19.108064,20.061065,20.823653,20.823653,21.024242,20.974731,20.615937,20.400457,20.119074,20.061065,21
6,11.907030,12.066509,13.053278,13.870714,14.662906,15.656504,16.553407,17.431312,18.143297,18.143297,18.331793,18.285223,17.948616,17.747235,17.485196,17.431312,18
7,10.747704,10.855698,11.551155,12.160519,12.777028,13.582372,14.336061,15.094723,15.722986,15.722986,15.891038,15.849455,15.550149,15.372176,15.141890,15.094723,16
8,10.061353,10.127468,10.567603,10.972933,11.400611,11.983944,12.552776,13.145118,13.648764,13.648764,13.785285,13.751438,13.509122,13.366175,13.182539,13.145118,14
9,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,10.556945,11
